Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [4]:
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

# If you're working locally:
else:
    DATA_PATH = '/Users/jasimrashid/Downloads/Datasets/ds15-predictive-modeling-challenge'
    # !pip install category_encoders==2.*
    # !pip install pandas-profiling==2.*

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.merge(pd.read_csv(DATA_PATH+'/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'/train_labels.csv'))
test = pd.read_csv(DATA_PATH+'/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'/sample_submission.csv')

train.shape, test.shape

((59400, 41), (14358, 40))

In [7]:
# Split train into train and val
train, val = train_test_split(train, train_size=0.80, test_size=.2, 
                              stratify=train['status_group'], random_state=42)
train.shape, val.shape, test.shape

((47520, 41), (11880, 41), (14358, 40))

In [33]:
submission = test[['id']].copy()

In [12]:
import numpy as np
def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    # Also create a "missing indicator" column, because the fact that
    # values are missing may be a predictive signal.
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
                       'gps_height', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()
            
    # Drop duplicate columns
    duplicates = ['quantity_group', 'payment_type']
    X = X.drop(columns=duplicates)
    
    # Drop recorded_by (never varies) and id (always varies, random)
    unusable_variance = ['recorded_by','id']
    X = X.drop(columns=unusable_variance)
    
    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    # Engineer feature: how many years from construction_year to date_recorded
    X['years'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['years'].isnull()
    
    # return the wrangled dataframe
    return X

train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [13]:
# The status_group column is the target
target = 'status_group'

# Get a dataframe with all train columns except the target
# train_features = train.drop(columns=[target,'region_code'])
train_features = train.drop(columns=[target,'region_code','ward'])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features

In [14]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [25]:
import category_encoders as ce
from scipy.stats import randint, uniform
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import RandomizedSearchCV

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42)
)

param_distributions = {
    'simpleimputer__strategy': ['mean','median','most_frequent'],
    'randomforestclassifier__n_estimators': (50,500),
    'randomforestclassifier__criterion': ['gini','entropy'],
    'randomforestclassifier__max_depth': [5,10,15,20,25,30,35,40,45,None],
    # 'randomforestclassifier__min_samples_split': [1,2,3]
    'randomforestclassifier__min_samples_leaf': [1,2,3],
    # 'randomforestclassifier__max_leaf_nodes'
    'randomforestclassifier__max_features': uniform(0,1)
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions = param_distributions,
    n_iter = 20,
    cv = 3,
    scoring = 'accuracy',
    verbose = 10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train, y_train);






Fitting 3 folds for each of 20 candidates, totalling 60 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   47.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 16.1min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 16.5min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 16.5min finished


In [26]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)

Best hyperparameters {'randomforestclassifier__criterion': 'entropy', 'randomforestclassifier__max_depth': 35, 'randomforestclassifier__max_features': 0.8123893460865124, 'randomforestclassifier__min_samples_leaf': 3, 'randomforestclassifier__n_estimators': 500, 'simpleimputer__strategy': 'mean'}
Cross-validation MAE -0.7993055555555556


In [27]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,11,18,15,1,2,9,6,7,14,10,16,0,19,8,3,13,4,12,17,5
mean_fit_time,181.518,7.30273,5.76844,165.264,11.171,14.8207,200.346,192.621,197.028,87.874,13.8091,16.1237,9.69615,19.3002,15.4394,13.7811,24.5177,13.8907,41.7854,1.92131
std_fit_time,2.17789,0.0642426,0.174178,2.09732,0.577684,0.0670942,1.52886,4.26454,0.876031,0.895931,0.434204,0.199549,0.666278,0.327495,1.28438,0.129224,0.261422,0.252296,0.845748,0.062291
mean_score_time,2.34873,0.452144,0.444961,2.91819,0.448871,0.427499,2.88936,2.54262,2.55326,2.03911,0.38797,0.455481,0.453108,0.424855,0.394637,0.448423,3.14233,2.03899,1.58509,0.28788
std_score_time,0.0446447,0.0135674,0.0177779,0.111401,0.0253786,0.0243018,0.0631689,0.165922,0.109718,0.0461916,0.0288886,0.0388,0.0390214,0.00796964,0.00515199,0.0134415,0.112575,0.0473912,0.114037,0.0113152
param_randomforestclassifier__criterion,entropy,gini,entropy,entropy,gini,entropy,entropy,gini,gini,gini,entropy,entropy,gini,entropy,gini,gini,entropy,gini,entropy,entropy
param_randomforestclassifier__max_depth,35,20,35,35,35,25,25,25,20,15,15,25,40,15,15,30,45,20,10,5
param_randomforestclassifier__max_features,0.812389,0.291016,0.140212,0.688322,0.438679,0.575291,0.864201,0.973985,0.980707,0.50675,0.620205,0.597784,0.394263,0.865381,0.807044,0.628037,0.0217714,0.0170376,0.241447,0.104839
param_randomforestclassifier__min_samples_leaf,3,1,3,2,2,3,2,3,2,3,1,1,1,3,3,1,1,3,2,3
param_randomforestclassifier__n_estimators,500,50,50,500,50,50,500,500,500,500,50,50,50,50,50,50,500,500,500,50
param_simpleimputer__strategy,mean,mean,most_frequent,median,most_frequent,most_frequent,most_frequent,most_frequent,most_frequent,median,median,median,median,most_frequent,mean,median,median,most_frequent,mean,median


In [28]:
pipeline = search.best_estimator_

In [29]:
from sklearn.metrics import mean_absolute_error

# X_test = test[features]
# y_test = test[target]

y_pred = pipeline.predict(X_test)
# mae = mean_absolute_error(y_test, y_pred)
# print(f'Test MAE: ${mae:,.0f}')

In [30]:
pipeline.score(X_val, y_val)

0.8124579124579124

In [36]:
# Create submission file
# submission = test[['id']].copy()
submission['status_group']=y_pred
submission.to_csv('Kaggle-0514-test1.csv', index=False)